In [110]:
import numpy as np
import pandas as pd
import json
import re
import datetime
from datetime import datetime as dt
from matplotlib import pyplot as plt
import sqlite3
from pandasql import sqldf

In [111]:
pd.set_option('display.max_rows', 11)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [112]:
users_df = pd.read_json('users.json', lines=True, convert_dates=False)
users_df.head()

,contributors_enabled,created_at,default_profile,default_profile_image,description,entities,favourites_count,follow_request_sent,followers_count,following,friends_count,geo_enabled,has_extended_profile,id,id_str,is_translation_enabled,is_translator,lang,listed_count,location,name,notifications,profile_background_color,profile_background_image_url,profile_background_image_url_https,profile_background_tile,profile_banner_url,profile_image_url,profile_image_url_https,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,protected,screen_name,statuses_count,time_zone,translator_type,url,utc_offset,verified
0,False,1417384037,True,False,"Official Twitter page of Alaska Governor Bill Walker; honored to serve AK w/ @LtGovMallott. RTs ≠ endorsements.Tweets signed ""-BW"" come directly from Governor.","{'description': {'urls': []}, 'url': {'urls': [{'display_url': 'gov.alaska.gov', 'expanded_url': 'http://gov.alaska.gov', 'indices': [0, 22], 'url': 'http://t.co/NO67LyZvUR'}]}}",115,False,5870,False,550,True,False,2915095729,2915095729,False,False,en,190,Alaska,Governor Bill Walker,False,C0DEED,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile_banners/2915095729/1417385137,http://pbs.twimg.com/profile_images/737753674466787329/rmi2xV_E_normal.jpg,https://pbs.twimg.com/profile_images/737753674466787329/rmi2xV_E_normal.jpg,1DA1F2,C0DEED,DDEEF6,333333,True,False,AkGovBillWalker,1642,None,none,http://t.co/NO67LyZvUR,NaN,True
1,False,1240239576,False,False,"U.S. Senator from Minnesota. Follows, Retweets, Replies ≠ endorsement","{'description': {'urls': []}, 'url': {'urls': [{'display_url': 'amyklobuchar.com', 'expanded_url': 'http://www.amyklobuchar.com', 'indices': [0, 22], 'url': 'http://t.co/mGGFBfggto'}]}}",24,False,170717,False,91751,False,False,33537967,33537967,False,False,en,2493,,Amy Klobuchar,False,026113,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,NaN,http://pbs.twimg.com/profile_images/1815362035/AJK_Twitter_normal.jpg,https://pbs.twimg.com/profile_images/1815362035/AJK_Twitter_normal.jpg,026113,F6D73F,89BCEE,3D3C3D,True,False,amyklobuchar,5567,Central Time (US & Canada),none,http://t.co/mGGFBfggto,-18000.0,True
2,False,1366837593,False,False,"Congressman for Maryland's 4th Congressional District, former Army Aviator, father and husband","{'description': {'urls': []}, 'url': {'urls': [{'display_url': 'anthonybrown.com', 'expanded_url': 'http://www.anthonybrown.com', 'indices': [0, 23], 'url': 'https://t.co/NYPWOVmoAx'}]}}",90,False,6067,False,369,True,True,1378000346,1378000346,False,False,en,170,"Mitchellville, MD",Anthony G. Brown,False,184466,http://pbs.twimg.com/profile_background_images/884068111/76c403a1354bd42620a38d317f6d0a23.jpeg,https://pbs.twimg.com/profile_background_images/884068111/76c403a1354bd42620a38d317f6d0a23.jpeg,True,https://pbs.twimg.com/profile_banners/1378000346/1479909083,http://pbs.twimg.com/profile_images/801128012573970433/2MGZIRal_normal.jpg,https://pbs.twimg.com/profile_images/801128012573970433/2MGZIRal_normal.jpg,6B2E13,FFFFFF,DDEEF6,333333,False,False,AnthonyBrownMD4,1878,Eastern Time (US & Canada),none,https://t.co/NYPWOVmoAx,-14400.0,True
3,False,1300739574,False,False,"Husband of 43 yrs, Dad of 4, Papaw of 6. Lifelong Arkansan. Governor of Arkansas. Former Congressman, Admin. of DEA, Under Sec. of Homeland Security.","{'description': {'urls': []}, 'url': {'urls': [{'display_url': 'governor.arkansas.gov', 'expanded_url': 'http://www.governor.arkansas.gov', 'indices': [0, 23], 'url': 'https://t.co/b38CUNlZHi'}]}}",440,False,25747,False,598,True,True,269992801,269992801,False,False,en,420,Arkansas,Gov. Asa Hutchinson,False,C0DEED,http://pbs.twimg.com/profile_background_images/378800000022012499/f440cc71434509f7036d04f5ed7cd67e.jpeg,https://pbs.twimg.com/profile_background_images/3788000

In [113]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548 entries, 0 to 547
Data columns (total 42 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   contributors_enabled                548 non-null    bool   
 1   created_at                          548 non-null    object 
 2   default_profile                     548 non-null    bool   
 3   default_profile_image               548 non-null    bool   
 4   description                         548 non-null    object 
 5   entities                            545 non-null    object 
 6   favourites_count                    548 non-null    int64  
 7   follow_request_sent                 548 non-null    bool   
 8   followers_count                     548 non-null    int64  
 9   following                           548 non-null    bool   
 10  friends_count                       548 non-null    int64  
 11  geo_enabled                         548 non-n

In [114]:
#Our created_at column contains different formats, both Unix timestamps as well as other formats. 
#Convert everything into a Unix format
#Then convert it to Int
#Then apply UTC conversion to get proper timestamp

def convert_to_unix_int(timestamp_str):
    try:
        # Attempt to convert as an integer (Unix timestamp)
        return int(timestamp_str)
    except ValueError:
        try:
            # Attempt to parse the different format
            timestamp = dt.strptime(timestamp_str, '%a %b %d %H:%M:%S %z %Y')
            return int(timestamp.timestamp())
        except ValueError:
            # Handle any other cases or return None for unsupported formats
            return None

# Apply the conversion function to the 'created_at' column
users_df['created_at'] = users_df['created_at'].apply(convert_to_unix_int)

# Now, the 'created_at' column contains Unix timestamps where possible
#Apply lambda function to convert to UTC Timestamp
users_df['created_at'] = users_df['created_at'].apply(lambda x: dt.utcfromtimestamp(x))

In [115]:
users_df.head()

,contributors_enabled,created_at,default_profile,default_profile_image,description,entities,favourites_count,follow_request_sent,followers_count,following,friends_count,geo_enabled,has_extended_profile,id,id_str,is_translation_enabled,is_translator,lang,listed_count,location,name,notifications,profile_background_color,profile_background_image_url,profile_background_image_url_https,profile_background_tile,profile_banner_url,profile_image_url,profile_image_url_https,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,protected,screen_name,statuses_count,time_zone,translator_type,url,utc_offset,verified
0,False,2014-11-30 21:47:17,True,False,"Official Twitter page of Alaska Governor Bill Walker; honored to serve AK w/ @LtGovMallott. RTs ≠ endorsements.Tweets signed ""-BW"" come directly from Governor.","{'description': {'urls': []}, 'url': {'urls': [{'display_url': 'gov.alaska.gov', 'expanded_url': 'http://gov.alaska.gov', 'indices': [0, 22], 'url': 'http://t.co/NO67LyZvUR'}]}}",115,False,5870,False,550,True,False,2915095729,2915095729,False,False,en,190,Alaska,Governor Bill Walker,False,C0DEED,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile_banners/2915095729/1417385137,http://pbs.twimg.com/profile_images/737753674466787329/rmi2xV_E_normal.jpg,https://pbs.twimg.com/profile_images/737753674466787329/rmi2xV_E_normal.jpg,1DA1F2,C0DEED,DDEEF6,333333,True,False,AkGovBillWalker,1642,None,none,http://t.co/NO67LyZvUR,NaN,True
1,False,2009-04-20 14:59:36,False,False,"U.S. Senator from Minnesota. Follows, Retweets, Replies ≠ endorsement","{'description': {'urls': []}, 'url': {'urls': [{'display_url': 'amyklobuchar.com', 'expanded_url': 'http://www.amyklobuchar.com', 'indices': [0, 22], 'url': 'http://t.co/mGGFBfggto'}]}}",24,False,170717,False,91751,False,False,33537967,33537967,False,False,en,2493,,Amy Klobuchar,False,026113,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,NaN,http://pbs.twimg.com/profile_images/1815362035/AJK_Twitter_normal.jpg,https://pbs.twimg.com/profile_images/1815362035/AJK_Twitter_normal.jpg,026113,F6D73F,89BCEE,3D3C3D,True,False,amyklobuchar,5567,Central Time (US & Canada),none,http://t.co/mGGFBfggto,-18000.0,True
2,False,2013-04-24 21:06:33,False,False,"Congressman for Maryland's 4th Congressional District, former Army Aviator, father and husband","{'description': {'urls': []}, 'url': {'urls': [{'display_url': 'anthonybrown.com', 'expanded_url': 'http://www.anthonybrown.com', 'indices': [0, 23], 'url': 'https://t.co/NYPWOVmoAx'}]}}",90,False,6067,False,369,True,True,1378000346,1378000346,False,False,en,170,"Mitchellville, MD",Anthony G. Brown,False,184466,http://pbs.twimg.com/profile_background_images/884068111/76c403a1354bd42620a38d317f6d0a23.jpeg,https://pbs.twimg.com/profile_background_images/884068111/76c403a1354bd42620a38d317f6d0a23.jpeg,True,https://pbs.twimg.com/profile_banners/1378000346/1479909083,http://pbs.twimg.com/profile_images/801128012573970433/2MGZIRal_normal.jpg,https://pbs.twimg.com/profile_images/801128012573970433/2MGZIRal_normal.jpg,6B2E13,FFFFFF,DDEEF6,333333,False,False,AnthonyBrownMD4,1878,Eastern Time (US & Canada),none,https://t.co/NYPWOVmoAx,-14400.0,True
3,False,2011-03-21 20:32:54,False,False,"Husband of 43 yrs, Dad of 4, Papaw of 6. Lifelong Arkansan. Governor of Arkansas. Former Congressman, Admin. of DEA, Under Sec. of Homeland Security.","{'description': {'urls': []}, 'url': {'urls': [{'display_url': 'governor.arkansas.gov', 'expanded_url': 'http://www.governor.arkansas.gov', 'indices': [0, 23], 'url': 'https://t.co/b38CUNlZHi'}]}}",440,False,25747,False,598,True,True,269992801,269992801,False,False,en,420,Arkansas,Gov. Asa Hutchinson,False,C0DEED,http://pbs.twimg.com/profile_background_images/378800000022012499/f440cc71434509f7036d04f5ed7cd67e.jpeg,https://pbs.twimg.c

In [116]:
pd.set_option('display.max_rows', None)  # Set the maximum rows to display
users_df['location'].value_counts()
#TODO: Clean the Location Column so similar cities like Washington, DC and Washington, D.C. are counted as the same cities.

                                  134
Washington, DC                     24
Washington, D.C.                    7
Oklahoma                            4
Oregon                              4
New York, NY                        3
Colorado                            3
Houston, TX                         3
Las Vegas, NV                       2
Chicago, IL                         2
Delaware                            2
Washington, D.C.                    2
Arizona, USA                        2
California                          2
Iowa                                2
Carson City, NV                     2
Arizona                             2
New York                            2
Louisiana, USA                      2
Utah                                2
Hartford, CT                        2
Tennessee                           2
United States                       2
Missouri                            2
Massachusetts                       2
Stockton, CA                        1
Saratoga Spr

In [117]:
#Let us scrape State information from Wikipedia
#We are going to store 5 columns - State, State Abbreviation, Population, Population Percentage, Abbreviation
#Import beautiful soup for scraping
import requests
from bs4 import BeautifulSoup

# The URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States"

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the table containing the list of states and territories
    table = soup.find('table', {'class': 'wikitable'})

    if table:
        # Initialize lists to store the state names and cities
        state_names = []
        state_abbr = []
        state_cap = []
        state_pop = []
        

        # Find all rows in the table (skip the header row)
        rows = table.find_all('tr')[2:]

        for row in rows:
            # Extract the state name (from the first column)
            state_name = row.find('th').text.strip()
            if '[B]' in state_name:
                cleaned_name = state_name.replace('[B]', '')
            else:
                cleaned_name = state_name
                
            
            #Extract the abbreviation of the state
            abbr = row.find('td').text.strip()

            # Extract the list of cities (from the second column, if available)
            city_column = row.find_all('td')
            city_names = city_column[1].text.strip()

                
            # Extract population from fourth column (from the second column, if available)
            pop_column = row.find_all('td')
            pop_num = pop_column[-8].text.strip()


            # Append state name and cities to the respective lists
            state_abbr.append(abbr)
            state_names.append(cleaned_name)
            state_cap.append(city_names)
            population_int = int(pop_num.replace(",", ""))
            state_pop.append(population_int)

    else:
        print("Table not found on the page.")

else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")
    

state_df = pd.DataFrame ({
    'State':state_names,
    'Abbreviation':state_abbr,
    'Capital':state_cap,
    'Population':state_pop
})
total_population = state_df['Population'].sum()
state_df['Percentage'] = (state_df['Population']/total_population)*100
state_df['Percentage'] = state_df['Percentage'].round(2)
state_df

,State,Abbreviation,Capital,Population,Percentage
0,Alabama,AL,Montgomery,5024279,1.52
1,Alaska,AK,Juneau,733391,0.22
2,Arizona,AZ,Phoenix,7151502,2.16
3,Arkansas,AR,Little Rock,3011524,0.91
4,California,CA,Sacramento,39538223,11.95
5,Colorado,CO,Denver,5773714,1.75
6,Connecticut,CT,Hartford,3605944,1.09
7,Delaware,DE,Dover,989948,0.30
8,Florida,FL,Tallahassee,21538187,6.51
9,Georgia,GA,Atlanta,10711908,3.24


In [118]:
#Now we're going to create a new column called States, which will check if each of the States matches 
#the description or location of each User. If it does, it will Place that state's name in the State column
#We will use this column over location for our analysis.
pd.reset_option('display.max_rows')
def find_state(row, states):
    for state in states['State']:
        if state in row['description'] or state in row['location']:
            return state
    for abbr in states['Abbreviation']:
        if abbr in row['description'] or abbr in row['location']:
            return states.loc[states['Abbreviation'] == abbr, 'State'].values[0]
    return None  # Return None if no state match is found

# Apply the find_state function to each row in the 'users' DataFrame and create a 'State' column
users_df['State'] = users_df.apply(find_state, args=(state_df,), axis=1)
users_df['State'].value_counts()

California        46
Washington        37
Texas             30
New York          26
Florida           25
Pennsylvania      19
Virginia          16
Michigan          16
Illinois          16
Ohio              16
North Carolina    15
Georgia           14
New Jersey        12
Maryland          11
Alabama           10
Arizona           10
Minnesota         10
South Carolina    10
Tennessee         10
Colorado          10
Missouri          10
Indiana            9
Louisiana          8
Iowa               8
Connecticut        8
Oregon             8
Massachusetts      8
Arkansas           7
Nevada             7
Oklahoma           7
Kansas             7
Hawaii             6
Utah               6
New Mexico         6
Kentucky           6
Wisconsin          5
Nebraska           5
New Hampshire      5
Maine              5
Mississippi        5
Alaska             4
South Dakota       4
Montana            4
Rhode Island       4
Delaware           4
Idaho              4
Vermont            3
Wyoming      

In [120]:
#This output seems satisfactory for our analysis. In real world situations, we have to be more discerning.
#Now let's look at the entities column and what it represents.
short_url = "http://t.co/NO67LyZvUR"

# Send an HTTP GET request to the short URL
response = requests.get(short_url)

# Check if the request was successful
if response.status_code == 200:
    # Get the final (long) URL from the response
    long_url = response.url
    print("Long URL:", long_url)
else:
    print("Failed to expand the short URL. Status code:", response.status_code)

Long URL: https://gov.alaska.gov/


In [ ]:
def expand_short_url(short_url):
    try:
        response = requests.get(short_url)
        if response.status_code == 200:
            return response.url
        else:
            return "Failed to expand"
    except requests.exceptions.RequestException:
        return "Failed to connect"
users_df['website'] = users_df['url'].apply(expand_short_url)